 ## Exercise 45 - Profile Update

In [2]:
inputPathWatched = "/data/students/bigdata-01QYD/ex_data/Ex45/data/watchedmovies.txt"
inputPathPreferences = "/data/students/bigdata-01QYD/ex_data/Ex45/data/preferences.txt"
inputPathMovies = "/data/students/bigdata-01QYD/ex_data/Ex45/data/movies.txt"
outputPath = "res_out_Ex44/"
threshold = 0.5

__input__ : 
* A textual file containing the list of movies watched by the users of a video on demand service: _userid,movieid,start-timestamp,end-timestamp_
* A second textual file containing the list of preferences for each user: _userid,movie-genre_
* A third textual file containing the list of movies with the associated information: _movieid,title,movie-genre_

__output__ : 
* Select for each user with a misleading profile (according to the same definition of Exercise #44) the list of movie genres that are not in his/her preferred genres and are associated with at least 5 movies watched by the user: _userid, selected (misleading) genre_

In [7]:
### PART 1 = EXERCISE 44 ###

# map into (movieid , userid)
watchedRDD = sc.textFile(inputPathWatched)
pairWatchedRDD = watchedRDD.map(lambda line : (line.split(",")[1] , line.split(",")[0]) )

# map into (movieid, genre)
movieRDD = sc.textFile(inputPathMovies)
pairMovieUserRDD = movieRDD.map(lambda line : (line.split(",")[0] , line.split(",")[2]) )

# join them
joinedMovieWatchedRDD = pairWatchedRDD.join(pairMovieUserRDD)

# join again
userGenresRDD = joinedMovieWatchedRDD.map(lambda pair: (pair[1][0], pair[1][1]))

# get preferences
preferencesRDD = sc.textFile(inputPathPreferences)
pairPreferencesRDD = preferencesRDD.map(lambda line : (line.split(",")[0] , line.split(",")[1]))

# cogroup the movie preference with the watched genres
joinedUserGenresRDD = userGenresRDD.cogroup(preferencesRDD)
joinedUserGenresRDD.mapValues(lambda elem: (list(elem[0]), list(elem[1]))).collect()

# remove the second line -> where does it come from?
# remove the second array
joinedUserGenresRDD = joinedUserGenresRDD.filter(lambda pair : pair[0] != "u")\
                                         .mapValues(lambda pair : pair[0])
joinedUserGenresRDD.collect()

def getMisleadingProfiles(pair):
    
    user = pair[0]
    genres = list(pair[1])
    
    preference = genres[0]
    
    # measure the total number of film watched
    counter = 0
    # measure the matched genres
    mathcedGenre = 0
    
    for i in range(1,len(genres)):
        
        if genres[i] == preference:
            mathcedGenre += 1
            
        counter += 1
        
    percentage = mathcedGenre/counter
    
    if percentage >= threshold:
        return True
    else:
        return False


misleadingUsersRDD = joinedUserGenresRDD.filter(getMisleadingProfiles)

In [8]:
misleadingUsersRDD.collect()

[('user2', <pyspark.resultiterable.ResultIterable at 0x7f3bba007750>),
 ('user1', <pyspark.resultiterable.ResultIterable at 0x7f3bba007090>)]

In [18]:
# we want to get (userid, misleading genre)
def misleadingGenres(pair):
    
    user = pair[0]
    likedGenre =  list(pair[1])[0]
    watchedGenres = list(pair[1])[1:]
    
    # create a dictionary
    # "genre" : watched_film
    genres_counter = {}
    
    # iterate over all the watched genres
    for i, genre in enumerate(watchedGenres):
        
        if genre != likedGenre:
            
            if genre not in genres_counter:
                genres_counter[genre] = 1
            else:
                genres_counter[genre] += 1
                
    # check the genres with more than 5 views
    aboveThreshGenres = []
    
    for key in genres_counter:
        
        if genres_counter[key] >= 5:
            aboveThreshGenres.append(user, key)
    
    return aboveThreshGenres

In [19]:
# Select the pairs (userid,misleading genre)
misleadingGenresRDD = misleadingUsersRDD.flatMap(misleadingGenres)

In [ ]:
misleadingGenresRDD.saveAsTextFile(outputPath)